To summarize the TC simulation process in TESLA: In notebooks 5-6 you go through the process of extracting historic or synthetic (Nakajo) TC tracks around your site and  performing MDA on these data sets. After performing vortex and SWAN modeling on your MDA TCs, you build and test a radial basis function to simulate TC outputs from new TC inputs in notebooks 7 and 8. 

For sites in Cascadia, TCs are not a relevant forcing. However, to avoid changing the infrastructure of TESLA, we want to create empty TC datasets that will allow us to bypass the steps of simulating TCs and essentially tell our simulation that TCs never occur.

#### Here are the files we need to create (are called in later notebooks):

##### in notebook 9 and 10: 

 - TCs_hist_r1.nc and TCs_hist_r1_params.nc
 
    - _, TCs_r1_params = db.Load_TCs_r1_hist()   # TCs historical parameters inside big radius
    

 - TCs_hist_r2.nc and TCs_hist_r2_params.nc
 
    - _, TCs_r2_params = db.Load_TCs_r2_hist()   # TCs historical parameters inside small radius
    

but rather than creating these files we just modify the notebooks by commenting out certain lines. 

##### in notebook 12:

 - TCs_sim_r2_params.nc
 
    - TCs_params = db.Load_TCs_r2_sim_params()    # TCs parameters (copula generated) 
    

 - TCs_sim_r2_RBF_output.nc
 
    - TCs_RBFs = db.Load_TCs_sim_r2_rbf_output()  # TCs numerical_IH-RBFs_interpolation output
    

 - TCs_synth_ProbsChange.nc
 
    - probs_TCs =  db.Load_TCs_probs_synth()      # TCs synthetic probabilities
    




In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# for user inputs
import tkinter as tk
from tkinter import filedialog
from tkinter import simpledialog

# pip
import numpy as np
import xarray as xr

# for input configuration
import sys
sys.path.append('../../..')  # Appending parent directory to Python path
from input_configuration import *

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..'))

# teslakit
from teslakit.database import Database

In [2]:
# Teslakit database

p_data = p_data_input
db = Database(p_data)

# make a new site
db.MakeNewSite(site_name)

Teslakit Site already exists at C:/Users/fergusonpc/OneDrive - CDM Smith/Trinnex/CSDP/Teslakit-SF/Teslakit-SF-OG/Teslakit-SF\sites\SF


### create and save TCs_hist_r1.nc, TCs_sim_r1_params.nc, TCs_hist_r2.nc, and TCs_sim_r2_params.nc

In [3]:
output = []

TCs_r1_tracks = xr.Dataset({'storm':output})

TCs_r1_params = xr.Dataset(
        {
            'pressure_min':(output),
            'pressure_mean':(output),
            'velocity_mean':(output),
            'gamma':(output),
            'delta':(output),
            'category':(output),
            'dmin_date':(output),
            'last_date':(output),
            'index_in':(output),
            'index_out':(output),
            'storm': (output)
        },
    )

TCs_r2_tracks = TCs_r1_tracks
TCs_r2_params = TCs_r2_tracks

# store data
db.Save_TCs_r1_hist(TCs_r1_tracks, TCs_r1_params)
db.Save_TCs_r2_hist(TCs_r2_tracks, TCs_r2_params)


TCs_r2_sim_params = xr.Dataset(
    {
        'pressure_mean':(output),
        'pressure_min':(output),
        'gamma':(output),
        'delta':(output),
        'velocity_mean':(output),
        'storm':(output),    
    },
)


print(TCs_r2_sim_params)

db.Save_TCs_r2_sim_params(TCs_r2_sim_params)


<xarray.Dataset>
Dimensions:        (delta: 0, gamma: 0, pressure_mean: 0, pressure_min: 0, storm: 0, velocity_mean: 0)
Coordinates:
  * pressure_mean  (pressure_mean) float64 
  * pressure_min   (pressure_min) float64 
  * gamma          (gamma) float64 
  * delta          (delta) float64 
  * velocity_mean  (velocity_mean) float64 
  * storm          (storm) float64 
Data variables:
    *empty*


### create and save TCs_sim_r2_RBF_output.nc

In [4]:

TCs_RBF_out = xr.Dataset(
    {
        'hs':(output),
        'tp':(output),
        'ss':(output),
        'twl':(output),
        'dir':(output),
        'mu':(output) ,
        'wind_speed':(output) ,
        'wind_dir':(output),
        'storm':(output)
    },
)
print(TCs_RBF_out)

# store data as xarray.Dataset
db.Save_TCs_sim_r2_rbf_output(TCs_RBF_out)


<xarray.Dataset>
Dimensions:     (dir: 0, hs: 0, mu: 0, ss: 0, storm: 0, tp: 0, twl: 0, wind_dir: 0, wind_speed: 0)
Coordinates:
  * hs          (hs) float64 
  * tp          (tp) float64 
  * ss          (ss) float64 
  * twl         (twl) float64 
  * dir         (dir) float64 
  * mu          (mu) float64 
  * wind_speed  (wind_speed) float64 
  * wind_dir    (wind_dir) float64 
  * storm       (storm) float64 
Data variables:
    *empty*


### create and save TCs_synth_ProbsChange.nc

In [5]:
category = [0,1,2,3,4,5]
m_count = np.empty((6,6))
m_count[:] = np.nan

categ_cp = xr.Dataset(
        {
            'category_change_count': (('category','category'), m_count[:,:]),
            'category_change_probs': (('category','category'), m_count[:,:]),
            'category_nochange_probs': (('category'), m_count[-1,:]),
            'category_change_cumsum': (('category','category'), m_count[:,:]),
        },
        coords = {
            'category': category
        }
    )
print(categ_cp)

db.Save_TCs_probs_synth(categ_cp)

<xarray.Dataset>
Dimensions:                  (category: 6)
Coordinates:
  * category                 (category) int32 0 1 2 3 4 5
Data variables:
    category_change_count    (category, category) float64 nan nan ... nan nan
    category_change_probs    (category, category) float64 nan nan ... nan nan
    category_nochange_probs  (category) float64 nan nan nan nan nan nan
    category_change_cumsum   (category, category) float64 nan nan ... nan nan
